In [1]:
import sys
sys.path.append('../')
# 这里是为了能偶访问到非当前文件夹中的包

In [2]:
import re
from typing import List
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

import sentence_transformers
import torch
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from config import *
from langchain.vectorstores import FAISS


d:\software\anaconda\anaconda3\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EMBEDDING_DEVICE = "cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu"
LLM_DEVICE = "cuda" if torch.cuda.is_available(
) else "mps" if torch.backends.mps.is_available() else "cpu"
#LLM_DEVICE,EMBEDDING_DEVICE are string
num_gpus = torch.cuda.device_count()
print("EMBEDDING_DEVICE: ", EMBEDDING_DEVICE)
print("LLM_DEVICE: ", LLM_DEVICE)
print("num_gpus: ", num_gpus)
print(LLM_DEVICE.lower().startswith("cuda"))

EMBEDDING_DEVICE:  cpu
LLM_DEVICE:  cpu
num_gpus:  0
False


In [4]:
embedding_model_dict = embedding_model_dict
llm_model_dict = llm_model_dict
EMBEDDING_DEVICE = EMBEDDING_DEVICE
LLM_DEVICE = LLM_DEVICE
num_gpus = num_gpus#GPU数量
large_language_model = init_llm
embedding_model=init_embedding_model

In [13]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[embedding_model], )
print('第一步加载成功')
embeddings.client = sentence_transformers.SentenceTransformer(
            embeddings.model_name,
            device=EMBEDDING_DEVICE,
            cache_folder=os.path.join(MODEL_CACHE_PATH,embeddings.model_name))
print('embedding模型加载成功')

第一步加载成功
embedding模型加载成功


相关包的安装：[LangChain:万能的非结构化文档载入详解（一） - 知乎 (zhihu.com)](https://zhuanlan.zhihu.com/p/624812261)



In [6]:
class ChineseTextSplitter(CharacterTextSplitter):
    def __init__(self, pdf: bool = False, **kwargs):
        super().__init__(**kwargs)
        self.pdf = pdf

    def split_text(self, text: str) -> List[str]:
        if self.pdf:
            text = re.sub(r"\n{3,}", "\n", text)
            text = re.sub('\s', ' ', text)
            text = text.replace("\n\n", "")
        sent_sep_pattern = re.compile(
            '([﹒﹔﹖﹗．。！？]["’”」』]{0,2}|(?=["‘“「『]{1,2}|$))') 
        sent_list = []
        for ele in sent_sep_pattern.split(text):
            if sent_sep_pattern.match(ele) and sent_list:
                sent_list[-1] += ele
            elif ele:
                sent_list.append(ele)
        return sent_list

In [7]:
filepath="1.pdf"
loader = UnstructuredFileLoader(filepath)
textsplitter = ChineseTextSplitter(pdf=True)
docs = loader.load_and_split(textsplitter)

In [8]:
docs[1]

Document(page_content='例如，车辆通常具有多模态的未来分布：它可以转弯、直行、减速、加速等。', metadata={'source': '1.pdf'})

In [9]:
# 向量存储
vector_store = FAISS.from_documents(docs, embeddings)

In [10]:
VECTOR_STORE_PATH=VECTOR_STORE_PATH
vector_store.save_local(f'../{VECTOR_STORE_PATH}/faiss_index')

In [11]:
loader = UnstructuredFileLoader(
    "1.pdf", strategy="fast", mode="elements"
)

In [12]:
docs = loader.load()